# Comparison 

In [1]:
from gsa import Gsa
from clonalg import Clonalg
from imperialistic import Imperialistic

import pandas as pd
import numpy as np
import time
import plotly.graph_objects as go
import copy

In [50]:
def plot_3D(b_low, b_up, f):
    x = np.arange(b_low, b_up, 0.2)
    y = np.arange(b_low, b_up, 0.2)
    x, y = np.meshgrid(x, y)
    z = f([x, y])
    fig = go.Figure(data=[go.Surface(z=z, x=x, y=y, colorscale='magma_r')])
    fig.show()
    
def show_values(values):
    x = [values[i][0] for i in range(len(values))]
    y = [values[i][1] for i in range(len(values))]
    mean = np.mean(values, axis=0)
    fig = go.Figure(data=go.Scatter(x=x, y=y, mode='markers', name='values', marker_color='rgba(152,0,0,.8)'))
    fig.add_trace(go.Scatter(x=[mean[0]], y=[mean[1]], mode='markers', name='average value'))
    fig.show()
    
def show_progress(values):
    x = [values[i][0] for i in range(len(values))]
    y = [values[i][1] for i in range(len(values))]
    mean = np.mean(values, axis=0)
    fig = go.Figure(data=go.Scatter(x=x, y=y, mode='lines+markers', name='progress', marker=dict( size=16,
                                                                                                    color=np.random.randn(500),
                                                                                                    colorscale='Viridis',
                                                                                                    showscale=True)))
    fig.add_trace(go.Scatter(x=[mean[0]], y=[mean[1]], mode='markers', name='average value'))
    fig.show()

In [3]:
def Matyas(var): #Expected minimum (0,0)
    x, y = var
    return 0.26*(x**2 + y**2) - 0.48*(x*y)

def Rastrigin(var): #Expected minimum (0,0)
    x, y = var
    return 20 + x**2 - 10*np.cos(2*np.pi*x) + y**2 - 10*np.cos(2*np.pi*y)

def Rosenbrock(var): #Expected minimum (0,0)
    x, y = var
    return 100*((y - x**2)**2) + (1-x)**2

def Booth(var): #Expected minimum (1,3)
    x, y = var
    return (x + 2*y - 7)**2 + (2*x + y -5)**2

In [45]:
def Matyas(var):
    x, y = var
    return 0.26*(x**2 + y**2) - 0.48*(x*y)

plot_3D(-10, 10, Matyas)

You can see from the graph that we expect values close to 0 for the minimum. And for the maximum, we expect values close to -5 or 5.

In [49]:
def Rastrigin(var): #Expected minimum (0,0)
    x, y = var
    return 20 + x**2 - 10*np.cos(2*np.pi*x) + y**2 - 10*np.cos(2*np.pi*y)


plot_3D(-5.12, 5.12, Rastrigin)

In [52]:
def Rosenbrock(var): #Expected minimum (0,0)
    x, y = var
    return 100*((y - x**2)**2) + (1-x)**2

plot_3D(-2, 2, Rosenbrock)

In [53]:
def Booth(var): #Expected minimum (1,3)
    x, y = var
    return (x + 2*y - 7)**2 + (2*x + y -5)**2

plot_3D(-10, 10, Booth)

#### First test
In the first test we will see what values the algorithm returns with a given number of populations and what time is needed. For each number of populations ten values will be generated and from this the average will be calculated.

In [42]:
def test1(alg, b_low, b_up, f, i, ifMin):
    start = time.time()
    opt = np.array([alg(N=i, b_low=-5, b_up=5, if_min = ifMin).optimize(f) for j in range(10)])
    end = time.time()
    x_mean = np.mean(opt[:,0])
    y_mean = np.mean(opt[:,1])
    return [alg.__name__, i, x_mean, y_mean, f([x_mean,y_mean]), (end-start)/10]

In [61]:
values = []
for i in range(10, 110, 10):
    values.append(test1(Gsa, -5, 5, Matyas, i, True))
    values.append(test1(Clonalg, -5, 5, Matyas, i, True))
    values.append(test1(Imperialistic, -5, 5, Matyas, i, True))
    
test1DF = pd.DataFrame(values, columns=['Algorithm','Population','x', 'y', 'Value','time'])
test1DF.groupby(['Population', 'Value']).min()

Algorithm         x         y      time
Population Value                                                    
10         2.119852e-03        Clonalg  0.222091  0.234278  0.010274
           3.175653e-02            Gsa -0.055739 -0.400279  0.005685
           4.293525e-02  Imperialistic  1.053834  1.001934  0.003491
20         6.806255e-05  Imperialistic  0.015379  0.029256  0.019347
           7.456532e-04        Clonalg  0.064312  0.011867  0.021939
           1.220112e-02            Gsa  0.303824  0.462860  0.028025
30         3.705621e-07  Imperialistic -0.000097 -0.001283  0.046675
           2.145128e-04        Clonalg  0.074668  0.069468  0.037133
           5.382587e-04            Gsa  0.093259  0.114079  0.087865
40         6.142147e-08  Imperialistic -0.000809 -0.000373  0.100431
           2.955964e-04        Clonalg -0.060893 -0.031952  0.049567
           3.587973e-02            Gsa  0.642874  0.870662  0.184641
50         3.724630e-08  Imperialistic -0.000436 -0.000742  0.163987
           7.593098e-04        Clonalg -0.140498 -0.129093  0.062932
           4.524446e-03            Gsa  0.050526 -0.083837  0.324651
60         8.735177e-08  Imperialistic -0.001457 -0.001493  0.258474
           2.214858e-04        Clonalg  0.075282  0.073165  0.080398
           2.528097e-03            Gsa -0.026006 -0.122105  0.494326
70         5.906304e-08  Imperialistic -0.000208  0.000278  0.333884
           1.709060e-04        Clonalg -0.035545 -0.011122  0.092350
           9.032739e-03            Gsa -0.444531 -0.484561  0.693709
80         4.533728e-09  Imperialistic -0.000319 -0.000245  0.476694
           6.801851e-05        Clonalg -0.032995 -0.020429  0.115875
           2.486847e-03            Gsa  0.144594  0.053023  0.978243
90         3.815347e-07  Imperialistic  0.002150  0.002870  0.707528
           7.917612e-04        Clonalg -0.100719 -0.132273  0.124567
           1.857555e-03            Gsa  0.085052  0.156448  1.281023
100        4.611521e-08  Imperialistic  0.001001  0.000754  0.827822
           8.123011e-05        Clonalg -0.044678 -0.037102  0.146032
           6.796734e-03            Gsa -0.089608  0.075252  1.578167